# Disciplina Tópicos Especiais II (C318)

###### Curso: Fundamentos de Machine Learning
###### Alunas: Luana Gribel, Mariana Helena, Sarah Brandão e Sinara Pimenta
###### Professor: Ricardo Augusto

End-to-End Machine Learning Project - Análise e Modelagem - Dataframe: Diabetes

-------------------------------------------------------------------------------------------------------------------------------

##### Objetivo: Prever se um paciente tem diabetes ou tendência a ter essa doença com bases em alguns dados médicos.

##### Regras de Negócio:
- É possível prever que um indivíduo pode possuir diabetes com base em dados médicos de forma automatizada, sem supervisão profissional?

- Qual o principal sintoma?

- Qual a principal doença recorrente em quem tem diabetes?

- Existe uma tendência maior em algum gênero ou faixa etária?

##### Tipo de problema: Classificação Binária

##### Enquadramento:
Aprendizagem supervisionada, porque a saída é conhecida.

## Importação de dados e bibliotecas

##### Bibliotecas utilizadas no projeto

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import klib
from sklearn.model_selection import StratifiedKFold 
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import confusion_matrix

##### Importação da base de dados

In [ ]:
# Especificando diretório para carregamento do arquivo (.csv)
df = pd.read_csv('diabetes.csv')

In [ ]:
# Informações sobre o dataframe (atributo info)
df.info()

##### Informações sobre o dataset:

Contexto: Dataset obtido a partir da coleta de dados com a aplicação de questionários para os pacientes do Sylhet Diabetes Hospital in Sylhet em Bangladesh.

Atributos:

| Atributo | Descrição | Valores |
|--- |--- |--- |
| Age | Faixa etária abordada na pesquisa | 20-65 |
| Gender | Gênero | Male/Female |
| Polyuria | Termo médico que faz referência ao sintoma de urinar em excesso (acima de 2,5 litros por dia) | Yes/No |
| Polydipsia | Polidipsia é um termo médico que define o sintoma caracterizado por excessiva sensação de sede | Yes/No |
| Sudden weight loss | Perda de peso repentina | Yes/No |
| Weakness | Fraqueza | Yes/No |
| Polyphagia | Polifagia é um sinal médico que significa fome excessiva e ingestão anormalmente alta de sólidos pela boca | Yes/No |
| Genital thrush | (Candidíase) é uma infecção fúngica causada por qualquer tipo do fungo Candida. | Yes/No |
| Visual blurring | Visão turva | Yes/No |
| Itching | Coceira | Yes/No |
| Irritability | Irritabilidade | Yes/No |
| Delayed healing | Cura demorada | Yes/No |
| Partial paresis | Paresia (Diminuição da força muscular)| Yes/No |
| Muscle stiffness | Rigidez muscilar| Yes/No |
| Alopecia | Perda de pelos do corpo | Yes/No |
| Obesity | Obesidade | Yes/No |
| Class | Classificação (Possui ou não diabetes) | Positive/Negative |

## Informações iniciais 

In [ ]:
# Descrição estatística do atributo Age
df.describe()

Conclusão:

In [ ]:
# Verificando as variáveis categóricas
generos = df["Gender"].value_counts()
print(generos)
print('----------------------------')
poliuria = df["Polyuria"].value_counts()
print(poliuria)
print('----------------------------')
polidipsia = df["Polydipsia"].value_counts()
print(polidipsia)
print('----------------------------')
perda_peso = df["sudden weight loss"].value_counts()
print(perda_peso)
print('----------------------------')
poliuria = df["weakness"].value_counts()
print(poliuria)
print('----------------------------')
poliuria = df["Polyphagia"].value_counts()
print(poliuria)
print('----------------------------')
poliuria = df["Genital thrush"].value_counts()
print(poliuria)
print('----------------------------')
poliuria = df["visual blurring"].value_counts()
print(poliuria)
print('----------------------------')
poliuria = df["Itching"].value_counts()
print(poliuria)
print('----------------------------')
poliuria = df["Irritability"].value_counts()
print(poliuria)
print('----------------------------')
poliuria = df["delayed healing"].value_counts()
print(poliuria)
print('----------------------------')
poliuria = df["partial paresis"].value_counts()
print(poliuria)
print('----------------------------')
poliuria = df["muscle stiffness"].value_counts()
print(poliuria)
print('----------------------------')
poliuria = df["Alopecia"].value_counts()
print(poliuria)
print('----------------------------')
poliuria = df["Obesity"].value_counts()
print(poliuria)

comentar da obesidade

Análise para o classificador não ficar inclinado a uma resposta:

In [ ]:
df["class"].value_counts()

In [ ]:
# Histograma plotado em cima da quantidade de pessoas
df['Age'].hist(bins = 50)
plt.xlabel('Idade')
plt.ylabel('Quantidade de pessoas')

In [ ]:
Estatistica distribuição da amostra.

In [ ]:
# Biblioteca para análise de dados (visualizações estatísticas)
import klib

In [ ]:
#Dispersão da variável Age
klib.dist_plot(df['Age'])

Saindo de uma estimativa empirica para uma densidade (histograma----func distribuição). Estatistica distribuição da população.
Densidade de probabilidade da população.
Inferencia estatística

skew: o quao assimetrico
Kurtosis: o quao achatada

In [ ]:
#Dispersão da variável Age, para pessoas que possuem diabetes
df_positive = df[(df['class']=='Positive')]
klib.dist_plot(df_positive['Age'])

In [ ]:
#Dispersão da variável Age, para pessoas que não possuem diabetes
df_negative = df[(df['class']=='Negative')]
klib.dist_plot(df_negative['Age'])

#### Preparação dos Dados - Manipulando features categóricas (ordinal encoder)

In [ ]:
df = df.replace(['Male','Female','Yes','No','Positive','Negative'],(1,0,1,0,1,0))
df.head()

## Amostragem 

Criação de conjuntos de dados de treino e teste. 

O método utilizado foi Train test split. -> guardar a proporção das classes

In [ ]:
#%% Método de Amostragem Aleatória Simples (Sklearn)
# Função do scikit-learn train_test_split
train_set, test_set = train_test_split(df, test_size = 0.2, random_state = 60)

In [ ]:
print(f"Tamanho conjunto de treino: {len(train_set)}")
print(f"Tamanho conjunto de teste: {len(test_set)}")

## Investigando Correlações

Será analisado o dataframe de treino. Objetivo de conhecer o dados e detectar as melhores variáveis para se trabalhar.

In [ ]:
#como realizar correlações com labels
#modelo de ml para identificar a importancia das variaveis categoricas

In [ ]:
# Fazendo a estimativa da matriz de correlação das variáveis do dataframe
correlation_matrix = train_set.corr(method='pearson')

In [ ]:
plt.figure(figsize=(15,10))
top_corr_features = correlation_matrix.index
sns.heatmap(correlation_matrix[top_corr_features],annot=True,cmap="RdYlGn")

In [ ]:
klib.corr_plot(train_set)

In [ ]:
klib.corr_plot(train_set, target='class')

### Modelagem

Um classificador random forest pode ser utilizado para computar a importância das features.

In [ ]:
X_train = train_set.iloc[:, 0:16].to_numpy()

y_train = train_set.loc[:, 'class'].to_numpy()

In [ ]:
feature_names = [f"feature {i}" for i in range(X_train.shape[1])]
forest = RandomForestClassifier(n_estimators=100)
forest.fit(X_train, y_train)

In [ ]:
importances = forest.feature_importances_
std = np.std([tree.feature_importances_ for tree in forest.estimators_], axis=0)

In [ ]:
importances

In [ ]:
import pandas as pd

forest_importances = pd.Series(importances, index=feature_names)

fig, ax = plt.subplots()
forest_importances.plot.bar(yerr=std, ax=ax)
ax.set_title("Feature importances using MDI")
ax.set_ylabel("Mean decrease in impurity")
fig.tight_layout()
#ranking de feature: o quao importante a variável é para realizar predições
#correlação diz que está correlacionado mas nao quer dizer que ter predição alta
#reta preta é a variabilidade,incerteza quanto a estimação do valor
#ordenar o gráfico

In [ ]:
#X_test
X_test = test_set.iloc[:, 0:16].to_numpy()
#Y_test
y_test = test_set.loc[:, 'class'].to_numpy()
#Train the model using the training sets  
var = forest.fit(X_train,y_train) 
y_pred=forest.predict(X_test)

In [ ]:
array_test = var.predict_proba(X_test)

In [ ]:
df_with_array_test = pd.DataFrame(array_test,columns=['%NaoTer','%Ter'])

In [ ]:
df_with_array_test

### Avaliação de Desempenho

Avaliação de desempenho com todos os dados (sem separação de treino e teste)

In [ ]:
#y_train_positive = df[(df['class']=='Negative')]
lin_mse = mean_squared_error(y_test, y_pred)
lin_rmse = np.sqrt(lin_mse)
lin_rmse

### Validação com Confusion Matrix

In [ ]:
tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()

In [ ]:
confusion_matrix(y_test, y_pred)

In [ ]:
tn

In [ ]:
fp

In [ ]:
fn

In [ ]:
tp